In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [27]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
learning_rate = 1e-2
batch_size = 100
epochs = 20

In [29]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310282  [   64/60000]
loss: 2.168090  [ 6464/60000]
loss: 1.801354  [12864/60000]
loss: 1.501800  [19264/60000]
loss: 1.151914  [25664/60000]
loss: 1.044501  [32064/60000]
loss: 0.996919  [38464/60000]
loss: 0.870345  [44864/60000]
loss: 0.885816  [51264/60000]
loss: 0.805749  [57664/60000]
Test Error: 
 Accuracy: 71.6%, Avg loss: 0.790424 

Epoch 2
-------------------------------
loss: 0.786708  [   64/60000]
loss: 0.849716  [ 6464/60000]
loss: 0.583189  [12864/60000]
loss: 0.780017  [19264/60000]
loss: 0.664374  [25664/60000]
loss: 0.636173  [32064/60000]
loss: 0.707156  [38464/60000]
loss: 0.685961  [44864/60000]
loss: 0.722726  [51264/60000]
loss: 0.630996  [57664/60000]
Test Error: 
 Accuracy: 78.3%, Avg loss: 0.630595 

Epoch 3
-------------------------------
loss: 0.555132  [   64/60000]
loss: 0.653928  [ 6464/60000]
loss: 0.438049  [12864/60000]
loss: 0.662344  [19264/60000]
loss: 0.582411  [25664/60000]
loss: 0.558802  [32064/600